In [11]:
import json
import time
import os
from pathlib import Path
from requests.exceptions import HTTPError
from web3 import Web3

from ethscan_main import POOL_ADDRESS


# === CONFIGURATION ===
INFURA_URL = "https://mainnet.infura.io/v3/3921fc62a7ce4cda98926f47409b3d19"
CONTRACT_ADDRESS = POOL_ADDRESS = "0xCBCdF9626bC03E24f779434178A73a0B4bad62eD"
ABI_FILE = "WETH_WBTC_pool.json"  # Load your contract ABI file
BLOCKS_FILE = "blocks_data.json"
TRANSACTIONS_FILE = "transactions.json"
METADATA_FILE = "processed_blocks.json"
BATCH_SIZE = 1000  # Number of transactions to process before writing to disk

# === CONNECT TO ETHEREUM NODE ===
w3 = Web3(Web3.HTTPProvider(INFURA_URL))

assert w3.is_connected(), "Web3 provider connection failed"


# === UTILITY FUNCTIONS ===
# Load processed metadata
def load_metadata():
    if Path(METADATA_FILE).exists():
        with open(METADATA_FILE, "r") as f:
            data = json.load(f)
            data["blocks"] = set(data.get("blocks", []))  # Ensure it's a set
            return data
    return {"blocks": set()}


# Save processed metadata
def save_metadata(metadata):
    metadata["blocks"] = list(metadata["blocks"])  # Convert set to list
    with open(METADATA_FILE, "w") as f:
        json.dump(metadata, f)


metadata = load_metadata()


# === LOAD CONTRACT ABI ===
def load_contract_abi(file_path):
    """Load the contract ABI from a JSON file."""
    try:
        with open(file_path, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        raise Exception(f"⚠️ ABI file '{file_path}' not found!")
    except json.JSONDecodeError:
        raise Exception(f"❌ Error parsing ABI JSON from '{file_path}'!")


contract_abi = load_contract_abi(ABI_FILE)
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=contract_abi)

In [ ]:
import json
import time
from hexbytes import HexBytes
from requests.exceptions import HTTPError


# Fetch latest 1,000 blocks
def fetch_blocks():
    latest_block = w3.eth.block_number
    blocks_to_fetch = [
        b
        for b in range(
            #latest_block - 999, latest_block + 1
            latest_block - 250, latest_block + 1
        )  # Adjusted to 3 blocks for testing
        if b not in metadata["blocks"]
    ]

    if not blocks_to_fetch:
        print("No new blocks to fetch.")
        return

    print(f"Fetching {len(blocks_to_fetch)} blocks: {blocks_to_fetch}")

    block_data = []

    for block_number in blocks_to_fetch:
        retries = 3
        while retries:
            try:
                block = w3.eth.get_block(block_number, full_transactions=True)

                # Log block hash
                print(f"Block {block_number} fetched with hash {block.hash.hex()}")

                # Process transactions
                transactions = block.transactions
                print(
                    f"Block {block_number} contains {len(transactions)} transactions."
                )

                # Store block data
                block_data.append(
                    {
                        "number": block.number,
                        "hash": block.hash.hex(),
                        "transactions": [
                            tx['hash'].hex() for tx in transactions
                        ],  # Convert transactions to strings
                    }
                )

                # Mark block as processed
                metadata["blocks"].add(block_number)
                break  # Exit retry loop

            except HTTPError as e:
                if e.response.status_code == 429:
                    print(
                        f"Rate limit hit, retrying block {block_number} in 2 seconds..."
                    )
                    time.sleep(2)
                    retries -= 1
                else:
                    print(f"Failed to fetch block {block_number}: {e}")
                    raise

    # Save block data if any blocks were fetched
    if block_data:
        print(f"Saving {len(block_data)} blocks to {BLOCKS_FILE}...")
        with open(BLOCKS_FILE, "w") as f:
            json.dump(
                block_data,
                f,
                default=lambda obj: obj.hex() if isinstance(obj, HexBytes) else obj,
            )

        save_metadata(metadata)
        print("Block data and metadata saved.")

    print("Fetching process completed.")

In [ ]:
import json
import time
from hexbytes import HexBytes
from requests.exceptions import HTTPError
from web3.datastructures import AttributeDict

TRANSACTIONS_FILE = "transactions.json"


def serialize_json(obj):
    """Convert Web3 objects to a JSON-friendly format."""
    if isinstance(obj, HexBytes):
        return obj.hex()
    if isinstance(obj, AttributeDict):
        return dict(obj)  # Convert AttributeDict to a regular dictionary
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")


def fetch_transactions():
    # Load saved block data
    try:
        with open(BLOCKS_FILE, "r") as f:
            blocks_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: {BLOCKS_FILE} not found. Run fetch_blocks() first.")
        return
    except json.JSONDecodeError:
        print(f"Error: {BLOCKS_FILE} is not valid JSON.")
        return

    transactions_data = []

    print(f"Processing {len(blocks_data)} blocks to extract transactions...")

    for block in blocks_data:
        block_number = block["number"]
        print(
            f"Processing block {block_number} with {len(block['transactions'])} transactions..."
        )

        for tx_hash in block["transactions"]:
            retries = 3
            while retries:
                try:
                    tx = w3.eth.get_transaction(tx_hash)

                    # Log the transaction
                    print(f"Fetched transaction {tx_hash} from block {block_number}")

                    # Store full transaction data
                    transactions_data.append(tx)

                    break  # Exit retry loop
                except HTTPError as e:
                    if e.response.status_code == 429:
                        print(
                            f"Rate limit hit, retrying transaction {tx_hash} in 2 seconds..."
                        )
                        time.sleep(2)
                        retries -= 1
                    else:
                        print(f"Failed to fetch transaction {tx_hash}: {e}")
                        raise

    # Save transaction data if any transactions were processed
    if transactions_data:
        print(f"Saving {len(transactions_data)} transactions to {TRANSACTIONS_FILE}...")
        with open(TRANSACTIONS_FILE, "w") as f:
            json.dump(transactions_data, f, default=serialize_json)

        print("Transaction data saved.")

    print("Transaction fetching process completed.")

In [2]:
import json
import time
from hexbytes import HexBytes
from requests.exceptions import HTTPError
from web3.datastructures import AttributeDict

TRANSACTIONS_FILE = "transactions.json"
BATCH_SIZE = 1000


def serialize_json(obj):
    """Convert Web3 objects to a JSON-friendly format."""
    if isinstance(obj, HexBytes):
        return obj.hex()
    if isinstance(obj, AttributeDict):
        return dict(obj)  # Convert AttributeDict to a regular dictionary
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")


def fetch_transactions():
    # Load saved block data
    try:
        with open(BLOCKS_FILE, "r") as f:
            blocks_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: {BLOCKS_FILE} not found. Run fetch_blocks() first.")
        return
    except json.JSONDecodeError:
        print(f"Error: {BLOCKS_FILE} is not valid JSON.")
        return

    transactions_data = []
    total_transactions = 0

    print(f"Processing {len(blocks_data)} blocks to extract transactions...")

    with open(TRANSACTIONS_FILE, "w") as f:
        for block in blocks_data:
            block_number = block["number"]
            print(
                f"Processing block {block_number} with {len(block['transactions'])} transactions..."
            )

            for tx_hash in block["transactions"]:
                retries = 3
                while retries:
                    try:
                        tx = w3.eth.get_transaction(tx_hash)

                        # Log the transaction
                        print(
                            f"Fetched transaction {tx_hash} from block {block_number}"
                        )

                        # Store full transaction data
                        transactions_data.append(tx)
                        total_transactions += 1

                        # Write transactions in batches
                        if total_transactions % BATCH_SIZE == 0:
                            for transaction in transactions_data:
                                json.dump(transaction, f, default=serialize_json)
                                f.write("\n")
                            transactions_data = []  # Clear the batch

                        break  # Exit retry loop
                    except HTTPError as e:
                        if e.response.status_code == 429:
                            print(
                                f"Rate limit hit, retrying transaction {tx_hash} in 2 seconds..."
                            )
                            time.sleep(2)
                            retries -= 1
                        else:
                            print(f"Failed to fetch transaction {tx_hash}: {e}")
                            raise

        # Write any remaining transactions
        if transactions_data:
            for transaction in transactions_data:
                json.dump(transaction, f, default=serialize_json)
                f.write("\n")

    print(f"Total transactions processed: {total_transactions}")
    print("Transaction fetching process completed.")

In [ ]:
# Run the processes
fetch_blocks()


In [3]:
fetch_transactions()

Processing 251 blocks to extract transactions...
Processing block 21986806 with 156 transactions...
Fetched transaction ba3e51e1d1041feee5d3b7164fd73d512b685da7fa408910ca5fc8c432f69ca4 from block 21986806
Fetched transaction a078b3d82b3966116eeb41d83dda2f3c27f048de95615b42b24744cd973ba968 from block 21986806
Fetched transaction b4ff50d8c16c09bd4c87bca7e2736126afd80fb1cc8891d502ab4dc737531e53 from block 21986806
Fetched transaction 32cc41a546d34346ef4ea44e8a8e6428fa8a850a386f7534d709b9cb58baba40 from block 21986806
Fetched transaction 69f85b84f9b3f7dc46a81d83001040e7e54fa486362ec4ede1e436aed074d704 from block 21986806
Fetched transaction db8b6635913f244bb30cdbc0d7de0f3e25af85610fcc73e2ba7f7b0d461a30cc from block 21986806
Fetched transaction 46df47ab1988ac3fcc13c1553db6553ab3ea674030e3539a9abc45fe748e1635 from block 21986806
Fetched transaction eb67e7d2441dc92003e749f048df64b3770b1ef4845b4ff5c220d560b7ae0d3d from block 21986806
Fetched transaction 68e951564e76be9df71e4d9b1418bf44f5460fe0

Web3RPCError: {'code': -32603, 'message': 'service temporarily unavailable'}

In [4]:
from web3 import Web3
from eth_abi import decode

# Mint function signature and selector (Uniswap V3 pool contract)
MINT_FUNCTION_SIGNATURE = "mint(address,uint256,uint256,bytes)"
MINT_SELECTOR = Web3.keccak(text=MINT_FUNCTION_SIGNATURE)[:4]  # First 4 bytes


def extract_mint_calls(transactions):
    """
    Extract and return transactions that call the Mint function.
    """
    mint_calls = []

    for tx in transactions:
        input_data = tx.get("input", "")
        if input_data and input_data.startswith(MINT_SELECTOR.hex()):
            try:
                decoded_params = decode(
                    [
                        "address",  # recipient
                        "uint256",  # tickLower
                        "uint256",  # tickUpper
                        "bytes",  # data
                    ],
                    bytes.fromhex(input_data[10:]),
                )  # Skip method selector

                mint_calls.append(
                    {
                        "blockNumber": tx["blockNumber"],
                        "from": tx["from"],
                        "to": tx["to"],
                        "gas": tx["gas"],
                        "gasPrice": tx["gasPrice"],
                        "recipient": decoded_params[0],
                        "tickLower": decoded_params[1],
                        "tickUpper": decoded_params[2],
                        "data": decoded_params[3].hex(),
                    }
                )
            except Exception as e:
                print(f"Error decoding Mint function: {e}")

    return mint_calls

In [5]:
m = extract_mint_calls()

TypeError: extract_mint_calls() missing 1 required positional argument: 'transactions'

In [ ]:
# Use the function signature directly
function_signature = "mint(address,int24,int24,uint128,bytes)"

# Compute the Keccak-256 hash and take the first 4 bytes (selector)
mint_selector = Web3.keccak(text=function_signature)[:4].hex()
print(f"✅ Extracted Mint() Function Selector: {mint_selector}")


# ✅ Load Already Processed Transactions (Checkpoints for Fast Resumption)
def load_processed_tx():
    """Returns a set of already processed transaction hashes to prevent duplicates."""
    if os.path.exists(MINT_CALLS_FILE):
        with open(MINT_CALLS_FILE, "r") as f:
            try:
                data = json.load(f)
                return set(entry["transactionHash"] for entry in data)
            except json.JSONDecodeError:
                print(f"⚠️ Warning: {MINT_CALLS_FILE} is corrupted. Resetting.")
                return set()
    return set()


processed_transactions = load_processed_tx()
# ✅ Load Transactions from Processed Blocks
def load_transactions(file_path):
    """Load stored transaction logs from `processed_blocks.json`."""
    with open(file_path, "r") as f:
        data = json.load(f)
        return data["logs"]


logs = load_transactions(PROCESSED_BLOCKS_FILE)

In [ ]:

# Load transaction data from file
def load_transactions(file_path="transactions.json"):
    with open(file_path, "r") as f:
        return json.load(f)


# Extract Mint function calls from transactions
def extract_mint_calls(transactions, contract_abi):
    mint_calls = []
    w3 = Web3()

    # Get the Mint function signature from the ABI
    contract = w3.eth.contract(abi=contract_abi)
    mint_function = contract.get_function_by_name("Mint")
    mint_selector = w3.keccak(text=mint_function.signature).hex()[:10]

    for tx in transactions:
        if tx.get("input", "").startswith(mint_selector):
            try:
                decoded = mint_function.decode_input(tx["input"])
                mint_calls.append({"tx_hash": tx["hash"], "params": decoded[1]})
            except Exception as e:
                print(f"Failed to decode Mint call in tx {tx['hash']}: {e}")

    return mint_calls


# Usage
transactions = load_transactions()
contract_abi = [...]  # Provide the actual ABI here
mint_calls = extract_mint_calls(transactions, contract_abi)

print(f"Found {len(mint_calls)} Mint function calls")

In [6]:
def extract_mint_calls(input_file="transactions.json", output_file="mint_calls.json"):
    # Load transaction data
    with open(input_file, "r") as f:
        transactions = json.load(f)

    mint_calls = {}
    # Use the function signature directly
    function_signature = "mint(address,int24,int24,uint128,bytes)"

    # Compute the Keccak-256 hash and take the first 4 bytes (selector)
    MINT_SELECTOR = Web3.keccak(text=function_signature)[:4].hex()

    for tx in transactions:
        tx_hash = tx.get("hash")
        input_data = tx.get("input", "")

        if input_data.startswith(MINT_SELECTOR):
            try:
                # Extract parameters (assuming first 4 bytes are function selector)
                raw_params = input_data[10:]  # Remove function selector
                params = Web3.toBytes(hexstr=raw_params)

                # Store result
                mint_calls[tx_hash] = {"raw_parameters": params.hex()}
            except Exception as e:
                print(f"Error decoding Mint call in {tx_hash}: {e}")

    # Save results to file
    with open(output_file, "w") as f:
        json.dump(mint_calls, f, indent=4)

    print(f"Extracted {len(mint_calls)} Mint function calls to {output_file}")


# Run the function
extract_mint_calls()

JSONDecodeError: Extra data: line 2 column 1 (char 1432)

In [7]:
import json
from web3 import Web3
from eth_abi import decode_abi

TRANSACTIONS_FILE = "transactions.json"
OUTPUT_FILE = "mint_calls.json"
MINT_FUNCTION_SIGNATURE = "mint(address,int24,int24,uint128,bytes)"
MINT_SELECTOR = Web3.keccak(text=MINT_FUNCTION_SIGNATURE)[:4].hex()


def decode_mint_parameters(input_data):
    """Decode the parameters of the mint function from the input data."""
    try:
        # Remove the function selector (first 4 bytes)
        data_without_selector = input_data[10:]  # Remove '0x' and selector
        # Decode the parameters
        params = decode_abi(
            ["address", "int24", "int24", "uint128", "bytes"],
            bytes.fromhex(data_without_selector),
        )
        return params
    except Exception as e:
        print(f"Error decoding parameters: {e}")
        return None


def process_transactions():
    """Process transactions to extract mint function calls."""
    try:
        with open(TRANSACTIONS_FILE, "r") as f:
            transactions = [json.loads(line) for line in f]
    except FileNotFoundError:
        print(f"Error: {TRANSACTIONS_FILE} not found.")
        return
    except json.JSONDecodeError:
        print(f"Error: {TRANSACTIONS_FILE} is not valid JSON.")
        return

    mint_calls = []
    for i, tx in enumerate(transactions):
        input_data = tx.get("input", "")
        if input_data.startswith(MINT_SELECTOR):
            params = decode_mint_parameters(input_data)
            if params:
                mint_calls.append(
                    {"transaction_hash": tx.get("hash"), "parameters": params}
                )

        # Write to file every 1,000 transactions
        if (i + 1) % 1000 == 0:
            with open(OUTPUT_FILE, "a") as out_file:
                for mint_call in mint_calls:
                    out_file.write(json.dumps(mint_call) + "\n")
            mint_calls = []  # Reset the list

    # Write any remaining mint calls
    if mint_calls:
        with open(OUTPUT_FILE, "a") as out_file:
            for mint_call in mint_calls:
                out_file.write(json.dumps(mint_call) + "\n")

    print("Processing completed.")

ImportError: cannot import name 'decode_abi' from 'eth_abi' (/home/ubuntu/.local/share/virtualenvs/closedblind-ylehWVqW/lib/python3.12/site-packages/eth_abi/__init__.py)

In [14]:
MINT_CALLS_FILE = "mint_calls.json"
BATCH_SIZE = 1000  # Number of transactions to process before writing to disk
# === FUNCTION SIGNATURE ===
function_signature = "mint(address,int24,int24,uint128,bytes)"
MINT_SELECTOR = Web3.keccak(text=function_signature)[:4].hex()


# === FUNCTION TO EXTRACT MINT CALLS ===
def extract_mint_calls(transactions_file, mint_calls_file, batch_size):
    if not Path(transactions_file).exists():
        raise FileNotFoundError(f"Transaction file '{transactions_file}' not found.")

    with open(transactions_file, "r") as infile, open(mint_calls_file, "w") as outfile:
        batch = []
        for line in infile:
            transaction = json.loads(line.strip())
            input_data = transaction.get("input", "")
            if input_data.startswith(MINT_SELECTOR):
                print("HERHEWHR")
                transaction_hash = transaction.get("hash")
                if transaction_hash:
                    batch.append({transaction_hash: input_data})

            if len(batch) >= batch_size:
                for mint_call in batch:
                    outfile.write(json.dumps(mint_call) + "\n")
                batch.clear()

        # Write any remaining transactions in the batch
        if batch:
            for mint_call in batch:
                outfile.write(json.dumps(mint_call) + "\n")


# === EXECUTE EXTRACTION ===
extract_mint_calls(TRANSACTIONS_FILE, MINT_CALLS_FILE, BATCH_SIZE)

In [ ]:
process_transactions()